# Plotting Poaching Density Using Dot Density and Histograms

In [1]:
# import packages
import pandas as pd
import geoplotlib
from geoplotlib.utils import read_csv
import warnings
warnings.filterwarnings('ignore')

In [2]:
# load the data
df = read_csv('Datasets/poaching_points_cleaned.csv')
df

DataAccessObject(['id_report', 'date_report', 'description', 'created_date', 'lat', 'lon'] x 268)

In [3]:
# load the data as a pandas dataframe
data = pd.read_csv('Datasets/poaching_points_cleaned.csv')
data.head()

,id_report,date_report,description,created_date,lat,lon
0,138,01/01/2005 12:00:00 AM,Poaching incident,2005/01/01 12:00:00 AM,-7.049359,34.841440
1,4,01/20/2005 12:00:00 AM,Poaching incident,2005/01/20 12:00:00 AM,-7.650840,34.480010
2,43,01/20/2005 12:00:00 AM,Poaching incident,2005/02/20 12:00:00 AM,-7.843202,34.005704
3,98,01/20/2005 12:00:00 AM,Poaching incident,2005/02/21 12:00:00 AM,-7.745846,33.948526
4,141,01/20/2005 12:00:00 AM,Poaching incident,2005/02/22 12:00:00 AM,-7.876673,33.690167


In [4]:
# Plot each row of our dataset as a single point on the map using a dot density layer
geoplotlib.dot(df)
geoplotlib.show()

In [5]:
# Visualize the density
geoplotlib.hist(df, binsize=20)
geoplotlib.show()

In [6]:
# Create a Voronoi plot
geoplotlib.voronoi(df, cmap='Blues_r', max_area=1e5, alpha=255)
geoplotlib.show()

# Creating a Choropleth Plot with GeoJSON Data

In [7]:
import json
from geoplotlib.colors import ColorMap
from geoplotlib.utils import BoundingBox

In [8]:
# open the GeoJSON file and load it as a json object
with open('Datasets/us_states.json') as data:
    dataset = json.load(data)
    
    fourth_state = dataset.get('features')[3]
    
    # only showing one coordinate instead of all points
    fourth_state['geometry']['coordinates'] = fourth_state['geometry']['coordinates'][0][0]
    print(json.dumps(fourth_state, indent=4))

{
    "type": "Feature",
    "properties": {
        "GEO_ID": "0400000US05",
        "STATE": "05",
        "NAME": "Arkansas",
        "LSAD": "",
        "CENSUSAREA": 52035.477
    },
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            -94.042964,
            33.019219
        ]
    }
}


In [9]:
# Extract the names of all the states of the USA from the dataset
with open('Datasets/us_states.json') as data:
    dataset = json.load(data)
    
    states = [feature['properties']['NAME'] for feature in dataset.get('features')]
    
    print('Number of states:', len(states))
    print(states)

Number of states: 52
['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming', 'Puerto Rico']


In [10]:
# Create a GeoJSON plot using the geojson()
geoplotlib.geojson('Datasets/us_states.json')
geoplotlib.show()

In [11]:
# Rather than assigning a single value to each state, we want the darkness to represent the number of census areas
cmap = ColorMap('Reds', alpha=255, levels=10)

def get_color(properties):
    return cmap.to_color(properties['SENSUSAREA'], maxvalue=300000, scale='lin')

In [12]:
# visualize the different states
geoplotlib.geojson('Datasets/us_states.json', fill=True, color=get_color)
geoplotlib.geojson('Datasets/us_states.json', fill=False, color=[0, 0, 0, 255])
geoplotlib.set_bbox(BoundingBox.USA)
geoplotlib.show()

Traceback (most recent call last):
  File "C:\Users\Hevans\Anaconda3\lib\site-packages\geoplotlib\__init__.py", line 32, in _runapp
    app.start()
  File "C:\Users\Hevans\Anaconda3\lib\site-packages\geoplotlib\core.py", line 367, in start
    l.invalidate(self.proj)
  File "C:\Users\Hevans\Anaconda3\lib\site-packages\geoplotlib\layers.py", line 974, in invalidate
    self.painter.set_color(self.color(feature['properties']))
  File "<ipython-input-11-d22f7b83fda0>", line 5, in get_color
    return cmap.to_color(properties['SENSUSAREA'], maxvalue=300000, scale='lin')
KeyError: 'SENSUSAREA'


In [13]:
# use the f_tooltip argument to provide a tooltip displaying
# the name and census area value of the state currently hovered over
geoplotlib.geojson('Datasets/us_states.json', fill=True, color=get_color, f_tooltip=lambda properties: properties['NAME'] + ' - Census Areas: ' + str(properties['CENSUSAREA']))
geoplotlib.geojson('Datasets/us_states.json', fill=False, color=[0, 0, 0, 255])
geoplotlib.set_bbox(BoundingBox.USA)
geoplotlib.show()

Traceback (most recent call last):
  File "C:\Users\Hevans\Anaconda3\lib\site-packages\geoplotlib\__init__.py", line 32, in _runapp
    app.start()
  File "C:\Users\Hevans\Anaconda3\lib\site-packages\geoplotlib\core.py", line 367, in start
    l.invalidate(self.proj)
  File "C:\Users\Hevans\Anaconda3\lib\site-packages\geoplotlib\layers.py", line 974, in invalidate
    self.painter.set_color(self.color(feature['properties']))
  File "<ipython-input-11-d22f7b83fda0>", line 5, in get_color
    return cmap.to_color(properties['SENSUSAREA'], maxvalue=300000, scale='lin')
KeyError: 'SENSUSAREA'


# Visually Comparing Different Tile Providers

In [14]:
# Display the map with the default tile provider
geoplotlib.show()

In [15]:
# Use the tiles_provider method and provide the 'darkmatter' tiles
geoplotlib.tiles_provider('darkmatter')
geoplotlib.show()

In [16]:
# Use the attribution element of the tiles_provider argument object to provide a custom attribution
geoplotlib.tiles_provider({
    'url': lambda zoom, xtile, ytile: 'http://a.tile.openstreetmap.fr/hot/%d/%d/%d.png' % 
      (zoom, xtile, ytile),
    'tiles_dir': 'custom_tiles',
    'attribution': 'Custom Tiles Provider – Humanitarian map style'
})
geoplotlib.show()

# Plotting the Movement of an Aircraft with a Custom Layer

In [17]:
import pandas as pd

In [18]:
# load the data
dataset = pd.read_csv('Datasets/flight_tracking.csv')
dataset.head()

,hex_ident,altitude(feet),latitude,longitude,date,time,angle,distance(nauticalmile),squawk,ground_speed(knotph),track,callsign
0,40631C,14525,53.65947,-1.43819,2017/09/11,17:02:06.418,-120.77,11.27,6276.0,299.0,283.0,NaN
1,40631C,14525,53.65956,-1.43921,2017/09/11,17:02:06.875,-120.64,11.30,6276.0,299.0,283.0,NaN
2,40631C,14500,53.65979,-1.44066,2017/09/11,17:02:07.342,-120.43,11.32,6276.0,299.0,283.0,EZY63BT
3,40631C,14475,53.66025,-1.44447,2017/09/11,17:02:09.238,-119.94,11.40,6276.0,299.0,283.0,EZY63BT
4,40631C,14475,53.66044,-1.44591,2017/09/11,17:02:09.825,-119.75,11.43,6276.0,299.0,283.0,EZY63BT


In [19]:
# Rename the latitude and longitude columns to lat and lon
dataset = dataset.rename(index=str, columns={'latitude':'lat', 'longitude':'lon'})

In [20]:
# Combine date and time into a timestamp
from datetime import datetime
def to_epoch(date, time):
    try:
        timestamp = round(datetime.strptime('{} {}'.format(date, time), '%Y/%m/%d %H:%M:%S.%f').timestamp())
        return timestamp
    except ValueError:
        return round(datetime.strptime('2017/09/11 17:02:06.418', '%Y/%m/%d %H:%M:%S.%f').timestamp())

In [21]:
# create a new column called timestamp that holds the Unix timestamp
dataset['timestamp'] = dataset.apply(lambda x: to_epoch(x['date'], x['time']), axis=1)

In [22]:
# filter the dataset for all the elements that are in the mentioned 
# time range and use each element of the filtered list to display it on the map
from geoplotlib.layers import BaseLayer
from geoplotlib.core import BatchPainter
from geoplotlib.colors import colorbrewer
from geoplotlib.utils import epoch_to_str, BoundingBox

In [23]:
class TrackLayer(BaseLayer):
    def __init__(self, dataset, bbox=BoundingBox.WORLD):
        self.data = dataset
        self.cmap = colorbrewer(self.data['hex_ident'], alpha=200)
        self.time = self.data['timestamp'].min()
        self.painter = BatchPainter()
        self.view = bbox
    def draw(self, proj, mouse_x, mouse_y, ui_manager):
        self.painter = BatchPainter()
        df = self.data.where((self.data['timestamp'] > self.time) 
			      & (self.data['timestamp'] <= self.time + 180))
        for element in set(df['hex_ident']):
            grp = df.where(df['hex_ident'] == element)
            self.painter.set_color(self.cmap[element])
            x, y = proj.lonlat_to_screen(grp['lon'], grp['lat'])
            self.painter.points(x, y, 15, rounded=True)
        self.time += 1
        if self.time > self.data['timestamp'].max():
            self.time = self.data['timestamp'].min()
        self.painter.batch_draw()
        ui_manager.info('Current timestamp: {}'.format(epoch_to_str(self.time)))
       
    # bounding box that gets used when the layer is created
    def bbox(self):
        return self.view

In [24]:
# Define a custom BoundingBox that focuses our view on this area
leeds_bbox = BoundingBox(north=53.8074, west=-3, south=53.7074 , east=0)

In [25]:
# convert any pandas DataFrame into a DataAccessObject class
from geoplotlib.utils import DataAccessObject
data = DataAccessObject(dataset)
geoplotlib.add_layer(TrackLayer(data, bbox=leeds_bbox))
geoplotlib.show()